<a href="https://colab.research.google.com/github/cicyfan/CodeSearchNet/blob/master/semantic-code-search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic Code Search

---
Team 16:	akb453@nyu.edu,	cf1311@nyu.edu,	emd543@nyu.edu 

## Introduction

The goal of this project is to understand how Neural Bag-of-Word can be used in sematic code search. Semantic code search is the task of retrieving relevant code given a natural language query. This requires connecting the natural language that is used to describe codes to the more formal language used to write the code. Here we present a description of Neural Bag-of-Word model, how it was implemented in a semantic code search problem and look at its performance. More specifically, we look at the Neural Bag-of-Word model proposed in work of Husain et. al. (source: [https://arxiv.org/pdf/1909.09436.pdf](https://arxiv.org/pdf/1909.09436.pdf))

## 1. Neural Bag-of-Word

We chose the Neural Bag-of-Word as our baseline model. Here, we adopt a top-down approach to describe the model - we start with the loss function, optimizer and gradually work our way towards the inputs. 

A Bag-Of-Words represents text as a vector of word features. Traditional BOW have used word occurance frequency or variants of term of document frequency as the word feature. However, developments in neural network and deep learning based NLP has lead to more powerful continuous vector representation of words and they often outperform the count based word representations. Specifically, Neural Bag-Of-Words excel at keyword matching, and was crucial in implementing search methods in our model.

### 1.1 The Adam Optimizer

Adam or the adaptive moment estimation, is an adaptive learning rate method that computes the individual learning rates for different parameters. Adam uses estimation of first and second moment of the gradient of the loss function to adapt the learning rate for each weight of a neural network. Since, the gradient of the loss function can be considered a random variable - the gradient is usually evaluated on some small random batch of data - the $n^{th}$ moment of the gradient can be calculated as $m_n = E[\,X^n]\,$ where $X$ is the gradient.

### 1.2 The Loss Function

The loss function of our model is the cosine similarity between the query and code token embeddings produced by the trainable query and code encoders. Both query and token embedding are 128 elements long vectors and the cosine similarity or the angle between a query and a token embedding can be determined using the dot product between these vectors.

#### 1.2.1 Dot Product

Dot product is used to "project" a vector onto a second vector (**see figure below**). This can be used to determine if the two vectors are pointing in the same direction (the angle between them). If the dot product is positive then the angle between them is less than $90^\circ$.

![](https://drive.google.com/uc?id=1uh5msrJgxgkyUZTiPLuqZd_uNUQXKf8R)

![](https://drive.google.com/uc?id=1TcMbpkGke_df-6Zq6Fc2EZGRhOmrQ-wE)

![](https://drive.google.com/uc?id=1Q1wmYJYPaSrX1hmjJsmts7he3GkZdbmu)

source: http://tutorial.math.lamar.edu/Classes/CalcII/DotProduct.aspx

The dot products between many query and code token embeddings can be carried out placing the token embeddings of the queries and tokens in two separate matrices respectively and multiplying these matrices. Each of the two token embedding matrices will have the following dimensions - $samples \times d$ - where $d$ is the dimensionality of the token embedding space (in this case 128). Before the multiplication, we need to normalize the each embedding (vector) by the norm of each batch.

Since each batch consists of a numbers of code and query token embeddings - $(c_i,d_i)$ - the ground truths are located on the diagonal of the consine similarity matrix. The loss per sample query $d_i$ is the distance between the ground truth $c_i$ and the closest/most similar code $c_j$ where $i \neq j$. We want to train our encoders such that codes that are most relevant to each other will have almost identical token embeddings, that is, the loss (distance between) them will be as close to zero as possible.

In the following code snippet the loss per sample query is determined by first taking the maximum similartiy per sample by passing the consine similarity matrix (after replacing the diagonal with negative infinity) through ReLU. Then we take the distance between the ground truths (the diagonal of the cosine similarity matrix) and the closest (highest similarity) code, $c_j$. In our model, negative cosine similarity is not penalized, so we set the loss to 0 if $c_i - c_j \lt 0$ where $c_i$ is the ground truth.

![](https://drive.google.com/uc?id=1Rn3bQD5jFWwlewu1Jv7JJM9JeBwGX148)

source: [https://github.com/github/CodeSearchNet/blob/master/src/models/model.py](https://github.com/github/CodeSearchNet/blob/master/src/models/model.py)

Then we store the average loss per batch and compute the Mean Reciprocal Rank (MRR).

#### 1.2.2 Mean Reciprocal Rank

Mean Reciprocal Rank (MRR) is used to measure the performace of a system that returns a ranked list of answers to queries. For a single query the reciprocal rank is $\frac{1}{rank}$ where $rank$ is the position of the highest-ranked answer. For multiple queries, Q, the mean of all the reciprocal queries is usually used. The MRR is calculated as follows. $MRR = \frac{1}{Q}\sum_{i=1}^{Q}\frac{1}{rank_i}$

In this case, there are more than one relevant result so we take the sum of all the corresponding ranks and then calculate the reciprocal. The mean of all the reciprocal ranks are calculated after all queries have been processed.

![](https://drive.google.com/uc?id=1-c3mROTppUCcIDhy9MeJ01CotvsVvEwB)

We take the gradient of this loss function and use the Adam optimizer to train the query and code encoders. In order to avoid gradient explosion, the gradient is clipped by rescaling the gradient so that the norm is at most 1.

![](https://drive.google.com/uc?id=1Y16r559DZ8hkrmS8_UDWeKlQdAoJrgrt)

### 1.3 Code and Query Token Embeddings

In section 1.2 we mentioned that the loss function is the cosine similarity between the code and token embeddings. Embeddings (also known as distributed representation) is a technique for learning vector representations of words and sentences in such a way that similar entities have vector representations that are close to each other. For example, the word 'execute' can be represented as [0.12,-0.32,0.01] and the word 'run' can be represented as [0.12,-0.31,0.02]. Based on these vectors, we can estimate the distances and semantic relations between these two words.

![](https://drive.google.com/uc?id=1pRd8sSoCGYETHQ5s6g76a8aKwoCV7WyF)

![](https://drive.google.com/uc?id=1hF0IYw4EuHJgsD8aPCDqc0nigo-vGaRt)

In our model, these embeddings are the trainable parameters (**see picture above**) that are determined using optimizer so that the loss (distance) between relevant queries and codes in the training set is small. Before training, the dimensionality of the embeddings space is set to be 128 and the embeddings are initialzed using Xavier initialization (**see picture below**).

![](https://drive.google.com/uc?id=1vV4FbtYPHZ2XBKmOiA1MlxlVnAlArfpB)

source: [https://github.com/github/CodeSearchNet/blob/master/src/encoders/seq_encoder.py](https://github.com/github/CodeSearchNet/blob/master/src/encoders/seq_encoder.py)

The code and query (docstring of the code) need to be processed first so that embeddings can be generated. In our model, the raw code and documentation pairs are parsed using GitHub's universal parser - TreeSitter. Byte Pair Encoding is then used on the parsed dataset to generate the vocabulary of the training corpus. The optimizer uses this vocabulary and the loss function to determine the values of the embeddings.

#### 1.3.1 Byte Pair Encoding

Byte Pair Encoding (BPE) is a simple form of data compression, in which common byte pairs are replaced by a single byte. Specifically, the most common adjacent byte pair in the document is replaced by a byte that never appears in the message, and this relationship is stored in a pair table for subsequent expansion. This process is then iterated for the new most common byte pair, until no further compression is possible. Because this is inherently a multipass algorithm, the entire contents of the message must be stored in memory during compression. Files too large to be held in memory can still be compressed using BPE by partitioning the file into blocks, reading one block at a time into a buffer, and then compressing the buffer. Following transmission, the message is then expanded by reversing the process, using the pair table formed during compression.

An adapted form of Byte Pair Encoding is particularly useful in Natural Language Processing for word segmentation. BPE enables a fixed-size vocabulary of variable-length character sequences to represent an open-ended vocabulary, which can successfully process unexpected words. In this case, instead of merging byte pairs, we encode individual characters or sequences of characters. The initial vocabulary consists of the characters in the language's alphabet. After the document is tokenized into individual words, we iteratively replace the most frequent character pairs with a new symbol. This new symbol is therefore an n-gram, representing the n characters that have been merged into it. Because we never consider character pairs that cross between tokens, each symbol represents either a word or a portion of a word. The size of the vocabulary is therefore equal to the number of characters in the alphabet plus the number of merge operations, and thus is finite and controllable by the user.

### 1.4 Putting Everything Together

We have described all the different components of the model previously. In this section, we put everything together to create a complete picture of the model. First the code, documentation pairs in the training dataset are parsed and then using Byte Pair Encoding the vocabulary of the corpus is generated. The goal is to train/determine the code and query (docstring) embeddings so that the cosine similarity between relevant query and codes are high. We acheived this by using the Adam optmizer on the gradient of the loss function (cosine similarity). Once the model is trained, we determine the representations (vectors) of the codes and queries in the test dataset using the code and query embeddings trained previously and return the codes with representations closest to the query representation.

# 2. Running the Project

[Link to our Github Repo](https://github.com/cicyfan/CodeSearchNet)

## 2.1 Train the model
```
python train.py --model neuralbow
```

OR train on python only, which is what we used to reduce runtime



```
train.py --model neuralbow ..resources/saved_models ../resources/data/python/final/jsonl/train ../resources/data/python/final/jsonl/valid ../resources/data/python/final/jsonl/test
```




![alt text](https://drive.google.com/uc?id=1DlO_jkdXo2PCSrnaEtlsWppbUBuLb4O_)


### 2.1.1 Training process
![alt text](https://drive.google.com/uc?id=1wB5kGGC3tsTZ8LnmO3L5qG_S_mm1DF0j)

### 2.1.2 Training finished
![alt text](https://drive.google.com/uc?id=1P2Oe3JfU-SDHVG-knz30lA2-l2fvGqsw)

## 2.2 Predict

predict.py has been modified to run only on Python, bypassing other languages (**see picture below**).

![](https://drive.google.com/uc?id=1VavzPvNsLoCYjdg5_iPbZ2VMf1qptJt8)

```
python predict.py -r wandb/CodeSearchNet/0123456  # this is the org/project_name/run_id
```
![alt text](https://drive.google.com/uc?id=1Tqrb3zLio3g8N6_grai6uK1NSqxXDQch)


# 3. Results

After submitting our results to the leaderboard on wandb, we received a **NDCG average of 0.31078**. Unfortunately, since there is a backlog on the reviewers of the Github repo to approve pull requests, our result did not show up on the leaderboard before the project deadline. 

[Link to our wandb dashboard](https://app.wandb.ai/cicyfan/CodeSearchNet)

[Link to our Github pull request](https://github.com/github/CodeSearchNet/pull/173)

![awaiting review](https://drive.google.com/uc?id=1kgS9UObJDYsnNQZrYmtYwCaUivJkJbj8)
![wandb run](https://drive.google.com/uc?id=1nxvndLYnMDLzMzfzrglTfQAsmibvr0Hz)
![submit to benchmark](https://drive.google.com/uc?id=1COyexjGkSWGcSisdk9vSxJgVQVnQgBpT)
